# PopSim Data Preparation (Census Mode)

This notebook prepares all input files for PopulationSim using German Census grid data.

## Requirements

1. **GeoPackage** (`.gpkg`) - polygon defining your study area boundary
2. **Census data** (parquet or CSV) - 100m and 1km grid cells with population attributes
3. **MiD seed data** (CSV) - household and person survey data (`MiD2023_Haushalte.csv`, `MiD2023_Personen.csv`)

## Census Data Format

Census files must have cell IDs in the **first column** matching this format:
- **100m**: `CRS3035RES100mN{northing}E{easting}` (e.g., `CRS3035RES100mN2689100E4337000`)
- **1km**: `CRS3035RES1000mN{northing}E{easting}` (e.g., `CRS3035RES1000mN2689000E4337000`)

Coordinates are **EPSG:3035** (ETRS89-extended / LAEA Europe). All other columns become available as control totals.

## What Gets Generated

### Single Mode (`regiostar_split=False`)
- `popsim/data/geo_cross_walk.csv` - geographic hierarchy
- `popsim/data/control_totals_*.csv` - census data formatted as control totals
- `popsim/data/seed_persons.csv`, `seed_households.csv` - filtered MiD data
- `popsim/configs/controls.csv` - control definitions for PopSim

### RegioStar Split Mode (`regiostar_split=True`)
Creates separate folders for each RegioStaR17 value found in the study area:
- `popsim_regiostar_121/` - census AND MiD filtered to RegioStaR17=121
- `popsim_regiostar_125/` - census AND MiD filtered to RegioStaR17=125
- etc.

Each folder contains complete PopSim inputs filtered to that regional type. The controls file is edited once in `popsim/configs/_prep3_controls.csv` and copied to all folders.

## Configuration

**Edit all paths and settings below before running.**

In [7]:
# =============================================================================
# USER CONFIGURATION
# =============================================================================

# --- Paths (relative to this notebook) ---
inputs_dir = "inputs"      # Shared input files (MiD, census, geopackages)
popsim_dir = "popsim"      # Base PopSim folder (for controls template, settings)

# --- Study Area ---
geopackage_path = f"{inputs_dir}/outlineBS.gpkg"
geopackage_crs = None  # Set CRS if not embedded (None = auto-detect)

# --- Census Data (parquet or CSV) ---
census_100m_path = f"{inputs_dir}/cells_100m_with_gender_backf_binneds_happyorphans_with_aggs_regiostar.parquet"
census_1km_path = f"{inputs_dir}/cells_1km_with_binneds.parquet"

# Column containing number of households (run Step 1 with None to see options)
household_column = "Insgesamt_Haushalte_Groesse_des_privaten_Haushalts_100m-Gitter_adj"  # e.g., "Insgesamt_Haushalte_100m-Gitter"

# --- MiD Seed Data (semicolon-separated CSVs) ---
mid_households_path = f"{inputs_dir}/MiD2023_Haushalte.csv"
mid_persons_path = f"{inputs_dir}/MiD2023_Personen.csv"

# --- MiD Filtering (set to None to skip that filter) ---
kernwo = [1,2,3]        # Day of week: [2,3] or None to skip. 1=Mon, 2=Tue-Thu, 3=Fri, 4=Sat-Sun
regiostar17 = None   # Regional types: [121,123,124] or None to skip (used when regiostar_split=False)

# --- RegioStar Split Mode ---
# When True: Creates separate popsim folders for each RegioStaR17 value in the study area
# Each folder gets census AND MiD data filtered to that RegioStaR17 value
regiostar_split = True

# --- CSV Separators ---
census_csv_sep = ";"   # For input CSVs (ignored for parquet)
intermediate_sep = ";"  # For intermediate files (use ";" for German Excel)
# Note: Final PopSim files are always comma-separated

# --- Advanced ---
output_everything = False  # True = output all PopSim intermediates
seed_geography = "STAAT"   # Geography level for seed data (usually unchanged)

# =============================================================================
# END CONFIGURATION
# =============================================================================

## Step 1: Load Study Area and Filter Census

Loads your GeoPackage, filters census cells to the study area, and shows available columns.

In [4]:
import os
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

print("[Step 1/4] Loading study area and filtering census...")
print("=" * 60)

# Ensure output directories exist
os.makedirs(f"{popsim_dir}/data", exist_ok=True)
os.makedirs(f"{popsim_dir}/configs", exist_ok=True)

# Load GeoPackage
print(f"Loading GeoPackage: {geopackage_path}")
study_area = gpd.read_file(geopackage_path)

# Handle CRS
if study_area.crs is None and geopackage_crs:
    study_area = study_area.set_crs(geopackage_crs)
    print(f"  Set CRS to: {geopackage_crs}")
elif study_area.crs is None:
    raise ValueError("GeoPackage has no CRS. Please set geopackage_crs in configuration.")

# Transform to EPSG:3035 (Census CRS)
study_area_3035 = study_area.to_crs("EPSG:3035")
bounds = study_area_3035.total_bounds  # minx, miny, maxx, maxy
print(f"  Study area bounds (EPSG:3035): {bounds}")

# Parse cell ID to extract coordinates
def parse_cell_id_100m(cell_id):
    """Extract N,E coordinates from 100m cell ID like CRS3035RES100mN2689100E4337000"""
    match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

def parse_cell_id_1km(cell_id):
    """Extract N,E coordinates from 1km cell ID like CRS3035RES1000mN2689000E4337000"""
    match = re.match(r'CRS3035RES1000mN(\d+)E(\d+)', str(cell_id))
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

def get_1km_id_from_100m(cell_id_100m):
    """Convert 100m cell ID to corresponding 1km cell ID."""
    n, e = parse_cell_id_100m(cell_id_100m)
    if n is None:
        return None
    n_1km = (n // 1000) * 1000
    e_1km = (e // 1000) * 1000
    return f"CRS3035RES1000mN{n_1km}E{e_1km}"

# Load 100m census
print(f"\nLoading 100m census: {census_100m_path}")

if census_100m_path.endswith('.parquet'):
    import pyarrow.parquet as pq
    pf_100m = pq.ParquetFile(census_100m_path)
    print(f"  Total rows: {pf_100m.metadata.num_rows:,}")
    print(f"  Total columns: {pf_100m.metadata.num_columns}")
    
    print("  Filtering to study area (this may take a moment)...")
    filtered_chunks = []
    total_read = 0
    
    for batch in pf_100m.iter_batches(batch_size=100000):
        df_batch = batch.to_pandas()
        total_read += len(df_batch)
        
        coords = df_batch.iloc[:, 0].apply(parse_cell_id_100m)
        df_batch['_N'] = coords.apply(lambda x: x[0])
        df_batch['_E'] = coords.apply(lambda x: x[1])
        
        mask = (
            (df_batch['_N'] >= bounds[1]) & (df_batch['_N'] <= bounds[3]) &
            (df_batch['_E'] >= bounds[0]) & (df_batch['_E'] <= bounds[2])
        )
        df_filtered = df_batch[mask].drop(columns=['_N', '_E'])
        
        if len(df_filtered) > 0:
            filtered_chunks.append(df_filtered)
        
        if total_read % 500000 == 0:
            print(f"    Processed {total_read:,} rows...")
    
    census_100m = pd.concat(filtered_chunks, ignore_index=True)
else:
    print(f"  Loading CSV with separator: '{census_csv_sep}'")
    census_100m_full = pd.read_csv(census_100m_path, sep=census_csv_sep)
    print(f"  Total rows: {len(census_100m_full):,}")
    
    coords = census_100m_full.iloc[:, 0].apply(parse_cell_id_100m)
    census_100m_full['_N'] = coords.apply(lambda x: x[0])
    census_100m_full['_E'] = coords.apply(lambda x: x[1])
    
    mask = (
        (census_100m_full['_N'] >= bounds[1]) & (census_100m_full['_N'] <= bounds[3]) &
        (census_100m_full['_E'] >= bounds[0]) & (census_100m_full['_E'] <= bounds[2])
    )
    census_100m = census_100m_full[mask].drop(columns=['_N', '_E']).copy()

print(f"  Filtered to {len(census_100m):,} cells in bounding box")

# Fine filter: check actual intersection with study area polygon
print("  Performing precise polygon intersection...")
id_col_100m = census_100m.columns[0]

def cell_intersects_study_area(cell_id):
    n, e = parse_cell_id_100m(cell_id)
    if n is None:
        return False
    cell_geom = box(e, n, e + 100, n + 100)
    return study_area_3035.geometry.intersects(cell_geom).any()

sample_mask = census_100m[id_col_100m].sample(min(100, len(census_100m))).apply(cell_intersects_study_area)
if sample_mask.mean() > 0.9:
    print("  Bounding box is tight, skipping detailed intersection.")
else:
    mask = census_100m[id_col_100m].apply(cell_intersects_study_area)
    census_100m = census_100m[mask]
    print(f"  After polygon intersection: {len(census_100m):,} cells")

# Find likely household columns
# print(f"\n{'='*60}")
# print("SUGGESTED HOUSEHOLD COLUMNS (first 5 values):")
# print(f"{'='*60}")

# hh_keywords = ['haushalt', 'household', 'hh_', 'wohnung']
# suggested = []
# for col in census_100m.columns:
#     col_lower = col.lower()
#     if any(kw in col_lower for kw in hh_keywords):
#         suggested.append(col)

# if suggested:
#     # Show all columns without truncation
#     with pd.option_context('display.max_columns', None, 'display.width', None):
#         display(census_100m[suggested].head())
# else:
#     print("  No household-related columns found.")
with pd.option_context('display.max_columns', None, 'display.width', None):
    display(census_100m.head())

# print(f"\nTotal columns available: {len(census_100m.columns)}")
# print("Use census_100m.columns to see all column names.")

# Load 1km census
print(f"\n{'='*60}")
print(f"Loading 1km census: {census_1km_path}")

if census_1km_path.endswith('.parquet'):
    census_1km_full = pd.read_parquet(census_1km_path)
else:
    census_1km_full = pd.read_csv(census_1km_path, sep=census_csv_sep)
print(f"  Total rows: {len(census_1km_full):,}")

# Filter 1km by deriving from 100m cells
km_ids_needed = set(census_100m[id_col_100m].apply(get_1km_id_from_100m).dropna())
id_col_1km = census_1km_full.columns[1]  # Usually GITTER_ID_1km
census_1km = census_1km_full[census_1km_full[id_col_1km].isin(km_ids_needed)].copy()
print(f"  Filtered to {len(census_1km):,} 1km cells")

# Save filtered data as parquet
census_100m.to_parquet(f'{popsim_dir}/data/_census_100m_filtered.parquet', index=False)
census_1km.to_parquet(f'{popsim_dir}/data/_census_1km_filtered.parquet', index=False)
print(f"\nSaved filtered census to {popsim_dir}/data/_census_*_filtered.parquet")

print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"  100m cells in study area: {len(census_100m):,}")
print(f"  1km cells in study area: {len(census_1km):,}")
print(f"  MiD households: {mid_households_path}")
print(f"  MiD persons: {mid_persons_path}")
kernwo_list = kernwo if isinstance(kernwo, list) else ([kernwo] if kernwo else None)
regiostar17_list = regiostar17 if isinstance(regiostar17, list) else ([regiostar17] if regiostar17 else None)
print(f"  MiD filters: kernwo={kernwo_list}, regiostar17={regiostar17_list}")
print(f"  RegioStar split mode: {regiostar_split}")
print(f"\nSet 'household_column' in Configuration and re-run Step 1,")
print("or proceed to Step 2 if already set.")
print("\n[Step 1/4] Complete.")

[Step 1/4] Loading study area and filtering census...
Loading GeoPackage: inputs/outlineBS.gpkg
  Study area bounds (EPSG:3035): [4349047.63188537 3230733.57551223 4364792.73274846 3249763.51079757]

Loading 100m census: inputs/cells_100m_with_gender_backf_binneds_happyorphans_with_aggs_regiostar.parquet
  Total rows: 3,148,482
  Total columns: 570
  Filtering to study area (this may take a moment)...
    Processed 500,000 rows...
    Processed 1,000,000 rows...
    Processed 1,500,000 rows...
    Processed 2,000,000 rows...
    Processed 2,500,000 rows...
    Processed 3,000,000 rows...
  Filtered to 6,338 cells in bounding box
  Performing precise polygon intersection...
  After polygon intersection: 5,147 cells


,GITTER_ID_100m,Insgesamt_Bevoelkerung_Alter_in_10er-Jahresgruppen_100m-Gitter,Unter10_Alter_in_10er-Jahresgruppen_100m-Gitter,a10bis19_Alter_in_10er-Jahresgruppen_100m-Gitter,a20bis29_Alter_in_10er-Jahresgruppen_100m-Gitter,a30bis39_Alter_in_10er-Jahresgruppen_100m-Gitter,a40bis49_Alter_in_10er-Jahresgruppen_100m-Gitter,a50bis59_Alter_in_10er-Jahresgruppen_100m-Gitter,a60bis69_Alter_in_10er-Jahresgruppen_100m-Gitter,a70bis79_Alter_in_10er-Jahresgruppen_100m-Gitter,a80undaelter_Alter_in_10er-Jahresgruppen_100m-Gitter,Insgesamt_Bevoelkerung_Alter_in_5_Altersklassen_100m-Gitter,Unter18_Alter_in_5_Altersklassen_100m-Gitter,a18bis29_Alter_in_5_Altersklassen_100m-Gitter,a30bis49_Alter_in_5_Altersklassen_100m-Gitter,a50bis64_Alter_in_5_Altersklassen_100m-Gitter,a65undaelter_Alter_in_5_Altersklassen_100m-Gitter,AnteilAuslaender_Anteil_Auslaender_100m-Gitter,AnteilUeber65_Anteil_ueber_65_100m-Gitter,AnteilUnter18_Anteil_unter_18_100m-Gitter,AnteilAuslaenderAb18_Auslaenderanteil_ab18_100m-Gitter,Einwohner_Bevoelkerungszahl_100m-Gitter,Deutsche_ab18_Deutsche_Staatsangehoerige_ab18_100m-Gitter,durchschnFlaechejeBew_Durchschn_Flaeche_je_Bewohner_100m-Gitter,durchschnFlaechejeWohn_Durchschn_Flaeche_je_Wohnung_100m-Gitter,DurchschnHHGroesse_Durchschn_Haushaltsgroesse_100m-Gitter,durchschnMieteQM_Durchschn_Nettokaltmiete_100m-Gitter,durchschnMieteQM_Durchschn_Nettokaltmiete_Anzahl_der_Wohnungen_100m-Gitter,AnzahlWohnungen_Durchschn_Nettokaltmiete_Anzahl_der_Wohnungen_100m-Gitter,Durchschnittsalter_Durchschnittsalter_100m-Gitter,Eigentuemerquote_Eigentuemerquote_100m-Gitter,Insgesamt_Energietraeger_Energietraeger_100m-Gitter,Gas_Energietraeger_100m-Gitter,Heizoel_Energietraeger_100m-Gitter,Holz_Holzpellets_Energietraeger_100m-Gitter,Biomasse_Biogas_Energietraeger_100m-Gitter,Solar_Geothermie_Waermepumpen_Energietraeger_100m-Gitter,Strom_Energietraeger_100m-Gitter,Kohle_Energietraeger_100m-Gitter,Fernwaerme_Energietraeger_100m-Gitter,kein_Energietraeger_Energietraeger_100m-Gitter,Insgesamt_Bevoelkerung_Familienstand_100m-Gitter,Ledig_Familienstand_100m-Gitter,Verheiratet_Familienstand_100m-Gitter,Verwitwet_Familienstand_100m-Gitter,Geschieden_Familienstand_100m-Gitter,EingetrLebenspartnerschaft_Familienstand_100m-Gitter,EingetrLebenspartVerstorben_Familienstand_100m-Gitter,EingetrLebenspartAufgehoben_Familienstand_100m-Gitter,OhneAngabe_Familienstand_100m-Gitter,Insgesamt_Wohnungen_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,unter30_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,30bis39_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,40bis49_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,50bis59_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,60bis69_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,70bis79_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,80bis89_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,90bis99_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,100bis109_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,110bis119_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,120bis129_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,130bis139_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,140bis149_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,150bis159_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,160bis169_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,170bis179_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,180undmehr_Flaeche_der_Wohnung_10m2_Intervalle_100m-Gitter,Insgesamt_Gebaeude_Geb_Gebaeudetyp_Groesse_100m-Gitter,FreiEFH_Geb_Gebaeudetyp_Groesse_100m-Gitter,EFH_DHH_Geb_Gebaeudetyp_Groesse_100m-Gitter,EFH_Reihenhaus_Geb_Gebaeudetyp_Groesse_100m-Gitter,Freist_ZFH_Geb_Gebaeudetyp_Groesse_100m-Gitter,ZFH_DHH_Geb_Gebaeudetyp_Groesse_100m-Gitter,ZFH_Reihenhaus_Geb_Gebaeudetyp_Groesse_100m-Gitter,MFH_3bis6Wohnungen_Geb_Gebaeudetyp_Groesse_100m-Gitter,MFH_7bis12Wohnungen_Geb_Gebaeudetyp_Groesse_100m-Gitter,MFH_13undmehrWohnungen_Geb_Gebaeudetyp_Groesse_100m-Gitter,AndererGebaeudetyp_Geb_Gebaeudetyp_Groesse_100m-Gitter,Insgesamt_Ge


Loading 1km census: inputs/cells_1km_with_binneds.parquet
  Total rows: 212,758
  Filtered to 165 1km cells

Saved filtered census to popsim/data/_census_*_filtered.parquet

SUMMARY
  100m cells in study area: 5,147
  1km cells in study area: 165
  MiD households: inputs/MiD2023_Haushalte.csv
  MiD persons: inputs/MiD2023_Personen.csv
  MiD filters: kernwo=[1, 2, 3], regiostar17=None
  RegioStar split mode: True

Set 'household_column' in Configuration and re-run Step 1,
or proceed to Step 2 if already set.

[Step 1/4] Complete.


## Step 2: Generate Geo Crosswalk and Control Totals

Creates the geographic hierarchy and control totals from filtered census data:
- `geo_cross_walk.csv` - mapping ZENSUS100m → ZENSUS1km → STAAT → WELT
- `control_totals_*.csv` - one file per geography level

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import yaml
from unidecode import unidecode

print("[Step 2/4] Generating geo crosswalk and control totals...")
print("=" * 60)

if household_column is None:
    raise ValueError("household_column not set! Set it in Configuration and re-run Step 1.")

# Load filtered census from Step 1
census_100m = pd.read_parquet(f'{popsim_dir}/data/_census_100m_filtered.parquet')
census_1km = pd.read_parquet(f'{popsim_dir}/data/_census_1km_filtered.parquet')

print(f"Loaded {len(census_100m):,} 100m cells, {len(census_1km):,} 1km cells")

# Validate household column
if household_column not in census_100m.columns:
    raise ValueError(f"household_column '{household_column}' not found in census data.")

# Check household values
hh_values = census_100m[household_column]
if hh_values.isna().any():
    na_count = hh_values.isna().sum()
    print(f"WARNING: {na_count} cells have missing household values (will be set to 0)")
if (hh_values < 0).any():
    neg_count = (hh_values < 0).sum()
    print(f"WARNING: {neg_count} cells have negative household values")

# Helper to get 1km ID from 100m ID
def get_1km_from_100m(cell_id):
    """Convert 100m cell ID to corresponding 1km cell ID."""
    match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
    if match:
        n, e = int(match.group(1)), int(match.group(2))
        n_1km = (n // 1000) * 1000
        e_1km = (e // 1000) * 1000
        return f"CRS3035RES1000mN{n_1km}E{e_1km}"
    return None

# Standardize column names
def clean_col_name(name):
    return unidecode(name).replace(" ", "").replace(".", "").replace(",", "").replace("-", "_")

# Rename columns
census_100m.columns = [clean_col_name(c) for c in census_100m.columns]
census_1km.columns = [clean_col_name(c) for c in census_1km.columns]
household_column_clean = clean_col_name(household_column)

# Find the ID columns after cleaning
id_col_100m_clean = census_100m.columns[0]
id_col_1km_clean = [c for c in census_1km.columns if '1km' in c.lower()][0]

# Create geo_cross_walk (hierarchy: ZENSUS100m -> ZENSUS1km -> STAAT -> WELT)
print("\nCreating geo_cross_walk...")
geo_cross = pd.DataFrame()
geo_cross['ZENSUS100m'] = census_100m[id_col_100m_clean]
geo_cross['ZENSUS1km'] = geo_cross['ZENSUS100m'].apply(get_1km_from_100m)
geo_cross['STAAT'] = 1
geo_cross['WELT'] = 1

geo_cross.to_csv(f'{popsim_dir}/data/geo_cross_walk.csv', index=False)
print(f"  Created {popsim_dir}/data/geo_cross_walk.csv ({len(geo_cross)} rows)")

# Create control_totals for 100m (lowest level)
print("\nCreating control totals...")

# Geography names (hierarchy from lowest to highest)
geo_names = ['ZENSUS100m', 'ZENSUS1km', 'STAAT', 'WELT']

# Add geography columns (NO rename of household column - keep original name)
census_100m = census_100m.rename(columns={id_col_100m_clean: 'ZENSUS100m'})
census_100m['ZENSUS1km'] = census_100m['ZENSUS100m'].apply(get_1km_from_100m)
census_100m['STAAT'] = 1
census_100m['WELT'] = 1

# Suffix non-geo columns
for col in census_100m.columns:
    if col not in geo_names:
        census_100m.rename(columns={col: f"{col}_ZENSUS100m"}, inplace=True)

# The household column with suffix
household_column_suffixed = f"{household_column_clean}_ZENSUS100m"

census_100m = census_100m.fillna(0)
census_100m.to_csv(f'{popsim_dir}/data/control_totals_ZENSUS100m.csv', index=False)
print(f"  Created {popsim_dir}/data/control_totals_ZENSUS100m.csv")

# Create control_totals for 1km
census_1km = census_1km.rename(columns={id_col_1km_clean: 'ZENSUS1km'})
census_1km['STAAT'] = 1
census_1km['WELT'] = 1

for col in census_1km.columns:
    if col not in geo_names:
        census_1km.rename(columns={col: f"{col}_ZENSUS1km"}, inplace=True)

census_1km = census_1km.fillna(0)
census_1km.to_csv(f'{popsim_dir}/data/control_totals_ZENSUS1km.csv', index=False)
print(f"  Created {popsim_dir}/data/control_totals_ZENSUS1km.csv")

# Create control_totals for STAAT
staat_df = pd.DataFrame({'STAAT': [1], 'WELT': [1]})
staat_df.to_csv(f'{popsim_dir}/data/control_totals_STAAT.csv', index=False)
print(f"  Created {popsim_dir}/data/control_totals_STAAT.csv")

# Create control_totals for WELT (top level)
welt_df = pd.DataFrame({'WELT': [1]})
welt_df.to_csv(f'{popsim_dir}/data/control_totals_WELT.csv', index=False)
print(f"  Created {popsim_dir}/data/control_totals_WELT.csv")

# Create controls template
print("\nCreating controls template...")
controls_rows = []
other_100m_rows = []

total_hh_control = None

# Add 100m controls - separate household from others
for col in census_100m.columns:
    if col not in geo_names:
        target = f"{col}_target"
        row = {
            'target': target,
            'geography': 'ZENSUS100m',
            'seed_table': '',
            'importance': '',
            'control_field': col,
            'expression': ''
        }
        # Identify household control by matching the suffixed household column name
        if col == household_column_suffixed:
            total_hh_control = target
            row['seed_table'] = 'households'
            row['importance'] = 1000
            row['expression'] = '(households.H_GEW > 0) & (households.H_GEW < np.inf)'
            controls_rows.insert(0, row)  # Insert at beginning
        else:
            other_100m_rows.append(row)

# Add other 100m rows after household
controls_rows.extend(other_100m_rows)

# Add 1km controls
for col in census_1km.columns:
    if col not in geo_names:
        controls_rows.append({
            'target': f"{col}_target",
            'geography': 'ZENSUS1km',
            'seed_table': '',
            'importance': '',
            'control_field': col,
            'expression': ''
        })

controls_df = pd.DataFrame(controls_rows)
controls_df.to_csv(f'{popsim_dir}/configs/_prep3_controls.csv', index=False, sep=intermediate_sep)
print(f"  Created {popsim_dir}/configs/_prep3_controls.csv ({len(controls_df)} controls, sep='{intermediate_sep}')")

if total_hh_control is None:
    raise ValueError(f"Could not find household control column '{household_column_suffixed}'!")

print(f"  Household control: {total_hh_control}")

# Update settings.yaml
print("\nUpdating PopSim configuration...")
with open(f'{popsim_dir}/configs/settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

# Geographies from top to bottom: WELT -> STAAT -> ZENSUS1km -> ZENSUS100m
settings['geographies'] = ['WELT', 'STAAT', 'ZENSUS1km', 'ZENSUS100m']
settings['seed_geography'] = seed_geography
settings['total_hh_control'] = total_hh_control

# Update input tables
idx = next((i for i, t in enumerate(settings['input_table_list']) if t['tablename'] == 'geo_cross_walk'), None)
if idx is not None:
    settings['input_table_list'] = settings['input_table_list'][:idx + 1]

for geo in ['ZENSUS100m', 'ZENSUS1km', 'STAAT', 'WELT']:
    settings['input_table_list'].append({
        'tablename': f'{geo}_control_data',
        'filename': f'control_totals_{geo}.csv'
    })

# Update output tables
if output_everything:
    settings['output_tables'] = {'action': 'skip', 'tables': 'geo_cross_walk'}
else:
    settings['output_tables'] = {
        'action': 'include',
        'tables': ['expanded_household_ids', 
                   'summary_ZENSUS100m', 'summary_ZENSUS1km', 'summary_STAAT', 'summary_WELT',
                   f'summary_ZENSUS100m_{seed_geography}']
    }

# Update models - add sub_balancing for each geography level below seed
settings['models'] = [m for m in settings['models'] if 'sub_balancing' not in m]
idx = settings['models'].index('integerize_final_seed_weights')
# Add sub_balancing for intermediate geographies (ZENSUS1km) then lowest (ZENSUS100m)
settings['models'].insert(idx + 1, 'sub_balancing.geography=ZENSUS1km')
settings['models'].insert(idx + 2, 'sub_balancing.geography=ZENSUS100m')

with open(f'{popsim_dir}/configs/settings.yaml', 'w') as f:
    yaml.dump(settings, f, default_flow_style=False)
print(f"  Updated {popsim_dir}/configs/settings.yaml")

# Update verification.yaml
with open(f'{popsim_dir}/scripts/verification.yaml', 'r') as f:
    verify = yaml.safe_load(f)

verify['group_geographies'] = ['WELT', 'STAAT', 'ZENSUS1km', 'ZENSUS100m']
verify['seed_cols']['geog'] = seed_geography
verify['summaries'] = [
    'output/final_summary_ZENSUS100m.csv',
    'output/final_summary_ZENSUS1km.csv',
    'output/final_summary_STAAT.csv',
    'output/final_summary_WELT.csv',
    f'output/final_summary_ZENSUS100m_{seed_geography}.csv'
]

with open(f'{popsim_dir}/scripts/verification.yaml', 'w') as f:
    yaml.dump(verify, f, default_flow_style=False)
print(f"  Updated {popsim_dir}/scripts/verification.yaml")

# Summary
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
total_hh = census_100m[household_column_suffixed].sum()
print(f"  Geographic hierarchy: WELT -> STAAT -> ZENSUS1km -> ZENSUS100m")
print(f"  Geographic units: {len(census_100m):,} (100m), {len(census_1km):,} (1km)")
print(f"  Total households: {total_hh:,.0f}")
print(f"  Household column: {household_column_suffixed}")
print(f"  Controls defined: {len(controls_df)}")
print(f"  Intermediate separator: '{intermediate_sep}'")
print(f"\nNext: Edit {popsim_dir}/configs/_prep3_controls.csv to add expressions")
print("for the controls you want, then run Step 3.")
print("\n[Step 2/4] Complete.")

## Step 3: Process Controls, Integerize, and Create PopSim Folders

1. Edit `popsim/configs/_prep3_controls.csv` to add expressions for the controls you want (done ONCE)
2. Run this cell to:
   - Filter census data to only columns needed by controls with expressions
   - **Smart integerize** 100m control totals (preserves 1km sums using largest remainder method)
   - If `regiostar_split=False`: Create single popsim folder with seed files
   - If `regiostar_split=True`: Create multiple `popsim_regiostar_{value}/` folders, each with census AND MiD filtered by that RegioStaR17 value

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import shutil
import yaml
from unidecode import unidecode

print("[Step 3/4] Processing controls, integerizing, and creating PopSim folders...")
print("=" * 60)

# Normalize filter lists
kernwo_list = kernwo if isinstance(kernwo, list) else ([kernwo] if kernwo else None)
regiostar17_list = regiostar17 if isinstance(regiostar17, list) else ([regiostar17] if regiostar17 else None)

# Load controls (intermediate file - use configured separator)
print(f"Loading controls template (separator: '{intermediate_sep}')...")
controls_df_full = pd.read_csv(f'{popsim_dir}/configs/_prep3_controls.csv', sep=intermediate_sep)
print(f"  Loaded {len(controls_df_full)} total controls from _prep3_controls.csv")

# Filter to controls that have expressions (these are the ones actually used)
controls_df = controls_df_full[controls_df_full['expression'].notna() & (controls_df_full['expression'] != '')].copy()
print(f"  {len(controls_df)} controls have expressions (will be used)")

# Extract control_field values - these are the census columns we actually need
needed_control_fields = set(controls_df['control_field'].tolist())
needed_100m_cols = {c for c in needed_control_fields if c.endswith('_ZENSUS100m')}
needed_1km_cols = {c for c in needed_control_fields if c.endswith('_ZENSUS1km')}
print(f"  100m columns needed: {len(needed_100m_cols)}")
print(f"  1km columns needed: {len(needed_1km_cols)}")

# =============================================================================
# SMART INTEGERIZATION FUNCTIONS
# =============================================================================

def get_1km_parent(id_100m: str) -> str:
    """Convert 100m cell ID to its parent 1km cell ID."""
    match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(id_100m))
    if match:
        n, e = int(match.group(1)), int(match.group(2))
        n_1km = (n // 1000) * 1000
        e_1km = (e // 1000) * 1000
        return f'CRS3035RES1000mN{n_1km}E{e_1km}'
    return None

def largest_remainder_round(values: np.ndarray, target_sum: int) -> np.ndarray:
    """Distribute integer values using largest remainder method (Hamilton apportionment)."""
    if target_sum == 0 or len(values) == 0:
        return np.zeros(len(values), dtype=int)
    
    total = values.sum()
    if total == 0:
        return np.zeros(len(values), dtype=int)
    
    # Scale values to sum to target
    scaled = values * (target_sum / total)
    
    # Floor all values
    floored = np.floor(scaled).astype(int)
    
    # Calculate remainders and distribute deficit
    remainders = scaled - floored
    deficit = target_sum - floored.sum()
    
    if deficit > 0:
        indices = np.argsort(-remainders)[:deficit]
        floored[indices] += 1
    elif deficit < 0:
        indices = np.argsort(remainders)[:-deficit]
        floored[indices] -= 1
    
    return floored

def smart_integerize_column(df: pd.DataFrame, col: str, group_col: str = 'ZENSUS1km') -> pd.Series:
    """Integerize a single column, preserving 1km sums."""
    result = pd.Series(index=df.index, dtype=int)
    
    for group_id, group_df in df.groupby(group_col):
        values = group_df[col].values.astype(float)
        target = int(round(values.sum()))
        if target < 0:
            target = 0
        int_values = largest_remainder_round(values, target)
        result.loc[group_df.index] = int_values
    
    return result

def smart_integerize_census(df_100m: pd.DataFrame, cols_to_integerize: set, id_col: str) -> pd.DataFrame:
    """Smart integerize specified columns in 100m census data."""
    df = df_100m.copy()
    
    # Add 1km parent mapping
    df['_ZENSUS1km'] = df[id_col].apply(get_1km_parent)
    
    # Find which columns to integerize (intersection of needed and available)
    available_cols = set(df.columns)
    cols_to_process = cols_to_integerize.intersection(available_cols)
    
    if not cols_to_process:
        print("    No columns to integerize")
        df = df.drop(columns=['_ZENSUS1km'])
        return df
    
    print(f"    Integerizing {len(cols_to_process)} columns...")
    
    for i, col in enumerate(cols_to_process):
        if (i + 1) % 10 == 0 or i == 0:
            print(f"      Processing {i+1}/{len(cols_to_process)}: {col[:50]}...")
        df[col] = smart_integerize_column(df, col, '_ZENSUS1km')
    
    df = df.drop(columns=['_ZENSUS1km'])
    return df

# =============================================================================
# LOAD SEED DATA
# =============================================================================

print(f"\nLoading MiD seed data...")
print(f"  Households: {mid_households_path}")
print(f"  Persons: {mid_persons_path}")
seed_households_full = pd.read_csv(mid_households_path, sep=',')
seed_persons_full = pd.read_csv(mid_persons_path, sep=',')
print(f"  Loaded {len(seed_persons_full):,} persons, {len(seed_households_full):,} households")

# Apply kernwo filter globally (only if not None) - this is done once
if kernwo_list:
    print(f"\nApplying kernwo filter:")
    persons_before = len(seed_persons_full)
    if 'kernwo' in seed_persons_full.columns:
        seed_persons_full = seed_persons_full[seed_persons_full['kernwo'].isin(kernwo_list)]
        print(f"  kernwo {kernwo_list}: {persons_before:,} -> {len(seed_persons_full):,} persons")

# Essential columns
essential_cols = {'H_ID', 'H_GEW', 'HP_ID', 'P_ID', 'P_GEW'}
needed_cols = essential_cols.copy()

# Extract columns from expressions
pattern = r'\.(?P<col>[A-Za-z_][A-Za-z0-9_]*)'
for expr in controls_df['expression'].dropna():
    for match in re.finditer(pattern, str(expr)):
        needed_cols.add(match.group('col'))

print(f"\nColumns needed from expressions: {needed_cols - essential_cols}")

# Standardize column names helper
def clean_col_name(name):
    return unidecode(name).replace(" ", "").replace(".", "").replace(",", "").replace("-", "_")

# Compute the suffixed household column name (same logic as Step 2)
household_column_clean = clean_col_name(household_column)
household_column_suffixed = f"{household_column_clean}_ZENSUS100m"

# Helper function to process and save seed data for a specific folder
def create_popsim_folder(output_dir, census_100m_filtered, census_1km_filtered, 
                         seed_persons_filtered, seed_households_filtered,
                         apply_integerization=True):
    """Create a complete popsim folder with all required files."""
    
    # Create directory structure
    os.makedirs(f"{output_dir}/data", exist_ok=True)
    os.makedirs(f"{output_dir}/configs", exist_ok=True)
    os.makedirs(f"{output_dir}/scripts", exist_ok=True)
    os.makedirs(f"{output_dir}/output", exist_ok=True)
    
    # Helper to get 1km ID from 100m ID
    def get_1km_from_100m(cell_id):
        match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
        if match:
            n, e = int(match.group(1)), int(match.group(2))
            n_1km = (n // 1000) * 1000
            e_1km = (e // 1000) * 1000
            return f"CRS3035RES1000mN{n_1km}E{e_1km}"
        return None
    
    # Process census 100m
    census_100m_proc = census_100m_filtered.copy()
    census_100m_proc.columns = [clean_col_name(c) for c in census_100m_proc.columns]
    
    id_col_100m_clean = census_100m_proc.columns[0]
    
    # Geography names
    geo_names = ['ZENSUS100m', 'ZENSUS1km', 'STAAT', 'WELT']
    
    # Create geo_cross_walk
    geo_cross = pd.DataFrame()
    geo_cross['ZENSUS100m'] = census_100m_proc[id_col_100m_clean]
    geo_cross['ZENSUS1km'] = geo_cross['ZENSUS100m'].apply(get_1km_from_100m)
    geo_cross['STAAT'] = 1
    geo_cross['WELT'] = 1
    geo_cross.to_csv(f'{output_dir}/data/geo_cross_walk.csv', index=False)
    
    # Add geography columns
    census_100m_proc = census_100m_proc.rename(columns={id_col_100m_clean: 'ZENSUS100m'})
    census_100m_proc['ZENSUS1km'] = census_100m_proc['ZENSUS100m'].apply(get_1km_from_100m)
    census_100m_proc['STAAT'] = 1
    census_100m_proc['WELT'] = 1
    
    # Suffix non-geo columns
    for col in list(census_100m_proc.columns):
        if col not in geo_names:
            census_100m_proc.rename(columns={col: f"{col}_ZENSUS100m"}, inplace=True)
    
    census_100m_proc = census_100m_proc.fillna(0)
    
    # Smart integerize 100m control columns (only the ones needed)
    if apply_integerization:
        print(f"  Smart integerizing 100m control totals...")
        census_100m_proc = smart_integerize_census(census_100m_proc, needed_100m_cols, 'ZENSUS100m')
    
    census_100m_proc.to_csv(f'{output_dir}/data/control_totals_ZENSUS100m.csv', index=False)
    
    # Process census 1km
    census_1km_proc = census_1km_filtered.copy()
    census_1km_proc.columns = [clean_col_name(c) for c in census_1km_proc.columns]
    id_col_1km_clean = [c for c in census_1km_proc.columns if '1km' in c.lower()][0]
    
    census_1km_proc = census_1km_proc.rename(columns={id_col_1km_clean: 'ZENSUS1km'})
    census_1km_proc['STAAT'] = 1
    census_1km_proc['WELT'] = 1
    
    for col in list(census_1km_proc.columns):
        if col not in geo_names:
            census_1km_proc.rename(columns={col: f"{col}_ZENSUS1km"}, inplace=True)
    
    census_1km_proc = census_1km_proc.fillna(0)
    
    # Integerize 1km columns (simple rounding since no sub-cells)
    if apply_integerization:
        for col in needed_1km_cols:
            if col in census_1km_proc.columns:
                census_1km_proc[col] = census_1km_proc[col].round().astype(int)
    
    census_1km_proc.to_csv(f'{output_dir}/data/control_totals_ZENSUS1km.csv', index=False)
    
    # Create STAAT and WELT control totals
    staat_df = pd.DataFrame({'STAAT': [1], 'WELT': [1]})
    staat_df.to_csv(f'{output_dir}/data/control_totals_STAAT.csv', index=False)
    
    welt_df = pd.DataFrame({'WELT': [1]})
    welt_df.to_csv(f'{output_dir}/data/control_totals_WELT.csv', index=False)
    
    # Process seed data - filter to needed columns
    p_cols = list(needed_cols.intersection(seed_persons_filtered.columns))
    h_cols = list(needed_cols.intersection(seed_households_filtered.columns))
    
    seed_persons_out = seed_persons_filtered[p_cols].copy()
    seed_households_out = seed_households_filtered[h_cols].copy()
    
    # Add STAAT geography
    seed_persons_out['STAAT'] = 1
    seed_households_out['STAAT'] = 1
    
    # Save seed files (comma-separated for PopSim)
    seed_persons_out.to_csv(f'{output_dir}/data/seed_persons.csv', index=False)
    seed_households_out.to_csv(f'{output_dir}/data/seed_households.csv', index=False)
    
    # Copy controls.csv (only rows with expressions)
    controls_df.to_csv(f'{output_dir}/configs/controls.csv', index=False)
    
    # Copy and adapt settings.yaml from base popsim folder
    shutil.copy(f'{popsim_dir}/configs/settings.yaml', f'{output_dir}/configs/settings.yaml')
    
    # Copy other config files if they exist
    for config_file in ['logging.yaml']:
        src = f'{popsim_dir}/configs/{config_file}'
        if os.path.exists(src):
            shutil.copy(src, f'{output_dir}/configs/{config_file}')
    
    # Copy scripts
    for script_file in ['verification.yaml']:
        src = f'{popsim_dir}/scripts/{script_file}'
        if os.path.exists(src):
            shutil.copy(src, f'{output_dir}/scripts/{script_file}')
    
    # Copy run script
    run_script = f'{popsim_dir}/run_populationsim.py'
    if os.path.exists(run_script):
        shutil.copy(run_script, f'{output_dir}/run_populationsim.py')
    
    return {
        'cells_100m': len(census_100m_proc),
        'cells_1km': len(census_1km_proc),
        'households': census_100m_proc[household_column_suffixed].sum(),
        'seed_persons': len(seed_persons_out),
        'seed_households': len(seed_households_out)
    }

# =============================================================================
# MAIN LOGIC: Split by RegioStar or single folder
# =============================================================================

if regiostar_split:
    print(f"\n{'='*60}")
    print("REGIOSTAR SPLIT MODE")
    print(f"{'='*60}")
    
    # Load the census data (already spatially filtered in Step 1)
    census_100m_base = pd.read_parquet(f'{popsim_dir}/data/_census_100m_filtered.parquet')
    census_1km_base = pd.read_parquet(f'{popsim_dir}/data/_census_1km_filtered.parquet')
    
    # Get RegioStaR17 column name from 100m census (may vary in case)
    regiostar_col_100m = [c for c in census_100m_base.columns if c.lower() == 'regiostar17'][0]
    
    # Get unique RegioStaR17 values from census (these are the ones in the study area)
    unique_regiostar = sorted(census_100m_base[regiostar_col_100m].dropna().unique())
    print(f"\nFound {len(unique_regiostar)} unique RegioStaR17 values in study area: {unique_regiostar}")
    
    # Check MiD has RegioStaR17 column
    if 'RegioStaR17' not in seed_persons_full.columns:
        raise ValueError("MiD persons data does not have 'RegioStaR17' column!")
    if 'RegioStaR17' not in seed_households_full.columns:
        raise ValueError("MiD households data does not have 'RegioStaR17' column!")
    
    created_folders = []
    
    for rs_value in unique_regiostar:
        rs_int = int(rs_value)  # Convert from float if needed
        folder_name = f"popsim_regiostar_{rs_int}"
        print(f"\n--- Creating {folder_name} ---")
        
        # Filter census by RegioStaR17
        census_100m_rs = census_100m_base[census_100m_base[regiostar_col_100m] == rs_value].copy()
        
        # Get 1km cells that correspond to the filtered 100m cells
        def get_1km_from_100m_id(cell_id):
            match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
            if match:
                n, e = int(match.group(1)), int(match.group(2))
                n_1km = (n // 1000) * 1000
                e_1km = (e // 1000) * 1000
                return f"CRS3035RES1000mN{n_1km}E{e_1km}"
            return None
        
        id_col_100m = census_100m_base.columns[0]
        km_ids_needed = set(census_100m_rs[id_col_100m].apply(get_1km_from_100m_id).dropna())
        id_col_1km = census_1km_base.columns[1]
        census_1km_rs = census_1km_base[census_1km_base[id_col_1km].isin(km_ids_needed)].copy()
        
        # Filter MiD by RegioStaR17
        seed_persons_rs = seed_persons_full[seed_persons_full['RegioStaR17'] == rs_int].copy()
        seed_households_rs = seed_households_full[seed_households_full['RegioStaR17'] == rs_int].copy()
        
        print(f"  Census: {len(census_100m_rs)} 100m cells, {len(census_1km_rs)} 1km cells")
        print(f"  MiD: {len(seed_persons_rs):,} persons, {len(seed_households_rs):,} households")
        
        if len(census_100m_rs) == 0:
            print(f"  WARNING: No census cells for RegioStaR17={rs_int}, skipping!")
            continue
        
        if len(seed_persons_rs) == 0 or len(seed_households_rs) == 0:
            print(f"  WARNING: No MiD data for RegioStaR17={rs_int}, skipping!")
            continue
        
        # Create the folder (with smart integerization)
        stats = create_popsim_folder(
            folder_name, 
            census_100m_rs, 
            census_1km_rs, 
            seed_persons_rs, 
            seed_households_rs,
            apply_integerization=True
        )
        
        created_folders.append({
            'folder': folder_name,
            'regiostar17': rs_int,
            **stats
        })
        print(f"  Created {folder_name}/ with {stats['cells_100m']} cells, {stats['households']:.0f} target households")
    
    # Summary
    print(f"\n{'='*60}")
    print("SUMMARY - RegioStar Split Mode")
    print(f"{'='*60}")
    print(f"\nCreated {len(created_folders)} popsim folders:")
    for info in created_folders:
        print(f"  {info['folder']}: RS17={info['regiostar17']}, "
              f"{info['cells_100m']} cells, {info['households']:.0f} HH, "
              f"{info['seed_persons']:,} seed persons")

else:
    # Original single-folder behavior
    print(f"\n{'='*60}")
    print("SINGLE FOLDER MODE")
    print(f"{'='*60}")
    
    # Apply regiostar17 filter if specified (only in single mode)
    seed_persons = seed_persons_full.copy()
    seed_households = seed_households_full.copy()
    
    if regiostar17_list:
        print(f"\nApplying regiostar17 filter:")
        persons_before = len(seed_persons)
        households_before = len(seed_households)
        if 'RegioStaR17' in seed_persons.columns:
            seed_persons = seed_persons[seed_persons['RegioStaR17'].isin(regiostar17_list)]
        if 'RegioStaR17' in seed_households.columns:
            seed_households = seed_households[seed_households['RegioStaR17'].isin(regiostar17_list)]
        print(f"  regiostar17 {regiostar17_list}: {len(seed_persons):,} persons, {len(seed_households):,} households")
    
    print(f"\nFinal counts:")
    print(f"  Persons: {len(seed_persons):,}")
    print(f"  Households: {len(seed_households):,}")
    
    # Load census data
    census_100m = pd.read_parquet(f'{popsim_dir}/data/_census_100m_filtered.parquet')
    census_1km = pd.read_parquet(f'{popsim_dir}/data/_census_1km_filtered.parquet')
    
    # Create the folder (with smart integerization)
    stats = create_popsim_folder(
        popsim_dir, 
        census_100m, 
        census_1km, 
        seed_persons, 
        seed_households,
        apply_integerization=True
    )
    
    print(f"\nCreated (all comma-separated for PopSim):")
    print(f"  {popsim_dir}/data/seed_persons.csv ({stats['seed_persons']} rows)")
    print(f"  {popsim_dir}/data/seed_households.csv ({stats['seed_households']} rows)")
    print(f"  {popsim_dir}/configs/controls.csv ({len(controls_df)} controls)")

print("\n[Step 3/4] Complete.")

## Step 4: Validate and Run

Validates the setup and provides instructions for running PopSim.

In [ ]:
import os
import glob
import json
import yaml
import pandas as pd

print("[Step 4/4] Validating setup...")
print("=" * 60)

def validate_popsim_folder(folder_path, folder_name):
    """Validate a single popsim folder and return errors list."""
    errors = []
    
    required_files = [
        'data/geo_cross_walk.csv',
        'data/seed_persons.csv',
        'data/seed_households.csv',
        'data/control_totals_ZENSUS100m.csv',
        'data/control_totals_ZENSUS1km.csv',
        'data/control_totals_STAAT.csv',
        'data/control_totals_WELT.csv',
        'configs/settings.yaml',
        'configs/controls.csv',
    ]
    
    print(f"\nChecking {folder_name}...")
    for f in required_files:
        full_path = f"{folder_path}/{f}"
        if os.path.exists(full_path):
            size = os.path.getsize(full_path)
            print(f"  [OK] {f} ({size:,} bytes)")
        else:
            print(f"  [MISSING] {f}")
            errors.append(f"Missing: {f}")
    
    # Check controls
    controls_path = f"{folder_path}/configs/controls.csv"
    if os.path.exists(controls_path):
        try:
            controls = pd.read_csv(controls_path)
            empty = controls['expression'].isna().sum()
            if empty > 0:
                errors.append(f"{empty} controls missing expressions")
            else:
                print(f"  {len(controls)} controls, all have expressions")
        except Exception as e:
            errors.append(f"Error reading controls: {e}")
    
    # Check settings
    settings_path = f"{folder_path}/configs/settings.yaml"
    if os.path.exists(settings_path):
        try:
            with open(settings_path) as f:
                settings = yaml.safe_load(f)
            print(f"  Geographies: {settings.get('geographies')}")
            print(f"  Total HH control: {settings.get('total_hh_control')}")
        except Exception as e:
            errors.append(f"Error reading settings: {e}")
    
    return errors

# =============================================================================
# VALIDATION
# =============================================================================

all_errors = {}

if regiostar_split:
    # Find all popsim_regiostar_* folders
    regiostar_folders = sorted(glob.glob("popsim_regiostar_*"))
    
    if not regiostar_folders:
        print("\nWARNING: No popsim_regiostar_* folders found!")
        print("Run Step 3 first to create them.")
    else:
        print(f"\nFound {len(regiostar_folders)} RegioStar folders to validate")
        
        for folder in regiostar_folders:
            folder_errors = validate_popsim_folder(folder, folder)
            if folder_errors:
                all_errors[folder] = folder_errors
        
        # Summary
        print(f"\n{'='*60}")
        if all_errors:
            print("VALIDATION FAILED")
            for folder, errors in all_errors.items():
                print(f"\n  {folder}:")
                for e in errors:
                    print(f"    - {e}")
        else:
            print("VALIDATION PASSED")
            print(f"\n{len(regiostar_folders)} folders ready to run PopSim.")
            print("\nTo run all folders:")
            print("  for d in popsim_regiostar_*/; do")
            print("    echo \"Running $d...\"")
            print("    (cd \"$d\" && python run_populationsim.py)")
            print("  done")
else:
    # Single folder validation
    folder_errors = validate_popsim_folder(popsim_dir, popsim_dir)
    if folder_errors:
        all_errors[popsim_dir] = folder_errors
    
    # Summary
    print(f"\n{'='*60}")
    if all_errors:
        print("VALIDATION FAILED")
        for e in all_errors[popsim_dir]:
            print(f"  - {e}")
    else:
        print("VALIDATION PASSED")
        print(f"\nReady to run PopSim:")
        print(f"  cd {popsim_dir}")
        print("  conda activate popsim")
        print("  python run_populationsim.py")

print(f"{'='*60}")
print("\n[Step 4/4] Complete.")

## Utilities: Reset

Clean up generated files to start fresh.

In [ ]:
import os
import glob
import shutil

def reset(confirm=False, include_regiostar_folders=False):
    """Delete all generated files.
    
    Args:
        confirm: Set to True to actually delete files
        include_regiostar_folders: Set to True to also delete popsim_regiostar_* folders
    """
    files = [
        f'{popsim_dir}/data/geo_cross_walk.csv',
        f'{popsim_dir}/data/seed_persons.csv',
        f'{popsim_dir}/data/seed_households.csv',
        f'{popsim_dir}/data/control_totals_ZENSUS100m.csv',
        f'{popsim_dir}/data/control_totals_ZENSUS1km.csv',
        f'{popsim_dir}/data/control_totals_STAAT.csv',
        f'{popsim_dir}/data/control_totals_WELT.csv',
        f'{popsim_dir}/data/_census_100m_filtered.parquet',
        f'{popsim_dir}/data/_census_1km_filtered.parquet',
        f'{popsim_dir}/configs/controls.csv',
        f'{popsim_dir}/configs/_prep3_controls.csv',
    ]
    
    existing_files = [f for f in files if os.path.exists(f)]
    
    # Find RegioStar folders
    regiostar_folders = sorted(glob.glob("popsim_regiostar_*")) if include_regiostar_folders else []
    
    if not existing_files and not regiostar_folders:
        print("No files to delete.")
        return
    
    if existing_files:
        print("Files to delete:")
        for f in existing_files:
            print(f"  {f}")
    
    if regiostar_folders:
        print(f"\nRegioStar folders to delete ({len(regiostar_folders)}):")
        for f in regiostar_folders:
            print(f"  {f}/")
    
    if not confirm:
        cmd = "reset(confirm=True"
        if regiostar_folders:
            cmd += ", include_regiostar_folders=True"
        cmd += ")"
        print(f"\nRun {cmd} to delete.")
        return
    
    for f in existing_files:
        os.remove(f)
        print(f"Deleted: {f}")
    
    for folder in regiostar_folders:
        shutil.rmtree(folder)
        print(f"Deleted: {folder}/")
    
    print("\nReset complete.")

# Show what would be deleted
reset(confirm=False, include_regiostar_folders=True)